# **EECS 16ML Assignment: Traditional/Contextual Word Embedding**

Before you start, make sure you review the lecture slide and notes about word embedding.

In [ ]:
# All Import Statements Defined Here
# Note: Do not add to this list.
# ----------------

import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem.porter import PorterStemmer
# if error appears, uncomment the next line
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from collections import Counter
from collections import defaultdict

# ----------------

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess_text(review):
    # remove HTML contents.
    soup = BeautifulSoup(review, "html.parser")
    review = soup.get_text()

    # remove everything except lower/upper case letters using Regular Expressions.
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)

    # bring everything into lowercase.
    review = review.lower()

    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]

    return review


In [ ]:
# Reading the dataset and visualize the first ten data points
path_data = '/content/IMDB Dataset.csv'
imdb_data=pd.read_csv(path_data)
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


Before vectorizing

In [ ]:
X_raw = imdb_data['review']
y = imdb_data['sentiment']
# Clean the raw data
tqdm.pandas()
X = X_raw.progress_apply(preprocess_text)

100%|██████████| 50000/50000 [25:05<00:00, 33.21it/s]


In [ ]:
X

0        [one, review, mention, watch, oz, episod, hook...
1        [wonder, littl, product, film, techniqu, unass...
2        [thought, wonder, way, spend, time, hot, summe...
3        [basic, famili, littl, boy, jake, think, zombi...
4        [petter, mattei, love, time, money, visual, st...
                               ...                        
49995    [thought, movi, right, good, job, creativ, ori...
49996    [bad, plot, bad, dialogu, bad, act, idiot, dir...
49997    [cathol, taught, parochi, elementari, school, ...
49998    [go, disagre, previou, comment, side, maltin, ...
49999    [one, expect, star, trek, movi, high, art, fan...
Name: review, Length: 50000, dtype: object

### **Part 1: Word-Count Vectors**

**Explain bag of words approach**

Define `vectorize` and `vectorizeDF` below to implement the Bag of Words approach, then apply `vectorizeDF` on the first 5,000 reviews.

*hint: use `.value_counts()`*

In [ ]:
# For now, use only first 5000 reviews
reviews_5000 = X[:5000]

In [ ]:
# This function turns a list of words into a vector by counting the occurences of every word in the list
# Input: bagOfWords - a list containing all the words in a review
# Output: vectorOfWords - a Pandas series containing occurences of every word in the input list
def vectorize(bagOfWords):
    #TODO 
    return pd.Series(bagOfWords).value_counts()

# This function applies vectorize() and vectorize all lists of words included in a Pandas dataframe
# Input: reviews - a Pandas dataframe containing all the reviews in *list of words* form
# Output: X_vectorized - a Pandas dataframe containing vectorized form of all reviews
def vectorizeDF(reviews):
    #TODO
    return reviews.progress_apply(vectorize).fillna(0).astype(int)

In [ ]:
X_vectorized = vectorizeDF(reviews_5000)

Check if applying implemented `vectorize` and `vectorizeDF` to out lists of words produce correct length of vocabulary:

In [ ]:
len(set(reviews_5000.explode())) == len(X_vectorized.columns)

**explain n-grams**

In [ ]:
# This function turns a list of words into a list of 2-grams
# Input: bagOfWords - a list containing all the words in a review
# Output: bagOf2grams - a list of 2-grams: [(word1, word2), (word2, word3) ... ]
def bigram(bagOfWords):
    #TODO
    return [(bagOfWords[i], bagOfWords[i+1]) for i in range(len(bagOfWords)-1)]

In [ ]:
reviews_5000.apply(bigram)

**explain nltk**

In [ ]:
from nltk import ngrams

In [ ]:
# This function turns a list of words into a list of 6-grams
# Input: bagOfWords - a list containing all the words in a review
# Output: bagOf6grams - a list of 6-grams: [(word1, word2, word3, word4, word5, word6) ... ]
def sixgram(bagOfWords):
    #TODO
    return list(ngrams(bagOfWords, 6))

In [ ]:
reviews_5000.apply(sixgram)

In the real world, implementing Bag of Words from scratch can be computation-heavy, especially on large dataset with longer texts. Fortunately, `Scikit-learn` has implemented it in an efficient way for us with `CountVectorizer`. Now using `CountVectorizer`, redefine `vectorizeDF` and apply it on the whole dataset. Feel free to play with the  parameter `ngram_range`, default (1, 1), for different ngrams. Note that `CountVectorizer` has a method `get_features_name()` that shows all the word tokens after vectorized if you want to take a look.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Modify our review data into string sentence form
reviews = X.progress_apply(lambda x: ' '.join(x))

In [ ]:
# This function vectorize all texts included in a Pandas dataframe
# Input: reviews - a list containing all the reviews in *string sentence* form
# Output: X_vectorized - a numpy array containing vectorized form of all reviews
def vectorizeDF(reviews):
    #TODO
    vectorizer = CountVectorizer(ngram_range = (1, 2))
    X_vectorized = vectorizer.fit_transform(reviews)
    return X_vectorized.toarray()

In [ ]:
X_vectorized = vectorizeDF(reviews)

Then, we use this vectorized data to train a classifier and report accuracy:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size = 0.2)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)

print('\n Accuracy: ', accuracy_score(y_train, y_pred))

### **Part 2: TFIDF**

In [ ]:
# This function computes the term frequency (TF), i.e. the number of times a word appears in a review divded 
# by the total number of words in the review.
# Input: bagOfWords - a list containing all the words in a review
# Output: tfDict - a dictionary containing the TF scores of all the words in a review
def computeTF(bagOfWords):
    # Frequency of each word in a review
    wordDict = dict(Counter(bagOfWords))
    tfDict = {}
    N = len(bagOfWords)
    # TODO: compute term frequency
    # Hint: Don't forget to apply logarithm transformation 
    for word, count in wordDict.items():
            tfDict[word] = np.log(1+count / float(N))

    return tfDict

# This function computes the inverse document frequency (IDF), i.e. the number of reviews divided by the number 
# of reviews that contain the word w. 
# Input: reviews - a list containing all the reviews
# Output: idfDict - a dictionary containing the IDF score of all the words in all the reviews
def computeIDF(reviews):
    N = len(reviews)
    # create a dictionary with all the words as keys and 0 as value
    reviews = [dict(Counter(review)) for review in reviews]
    idfDict = dict.fromkeys(set().union(*(review.keys() for review in reviews)),0)
    # TODO: compute inverse document frequency
    # Hint: Don't forget to apply logarithm transformation 
    for document in reviews:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
        
    for word, val in idfDict.items():
        idfDict[word] = np.log(N / float(val))
    return idfDict

# This function computes the TF-IDF score of all the words in a review
# Input: bagOfWords - a list containing all the words in a review
#        idfDict - a dictionary containing the IDF score of all the words in all the reviews
# Output: tfidf - a dictionary containing the TF-IDF scores of all the words in a review
def computeTFIDF(bagOfWords, idfDict):
    tfidf = {}  
    tfDict = computeTF(bagOfWords)
    # TODO: compute tf-idf
    for word, val in tfDict.items():
        tfidf[word] = val * idfs[word]
  
    return tfidf

In [ ]:
idfs = computeIDF(X)
tfidf = X.progress_apply(computeTFIDF, idfDict=idfs)

100%|██████████| 50000/50000 [00:10<00:00, 4836.63it/s]


After calculating the TF-IDF scores of all words in all the reviews, we need to map each review to a vector of scores. Notice that the dimension of each vector is the length of total unique words. Run the following cell. This might take 5 mins to run. 

In [ ]:
def vectorized(tfidf_doc):
    vec = np.zeros(len(total_vocab))
    for word, val in tfidf_doc.items():
        ind = total_vocab.index(word)
        vec[ind] = val
    return vec

X_vectorized = list(tfidf.progress_apply(vectorized))


You have successfully transformed each review to a vector of real numbers. Congrats! 

Next, we need to apply the vectorized reviews to our sentimental classification problem. We choose one of the easiest models - Naive Bayes classifier to train.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size = 0.2)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)

print('\n Accuracy: ', accuracy_score(y_train, y_pred))

As you can see

### **Part 3: Co-Occurrence Matrix with a fixed context window**

Words co-occurrence matrix describes how words occur together that in turn captures the relationships between words. Words co-occurrence matrix is computed simply by counting how two or more words occur together in a given corpus. 

In [ ]:
# TODD 
# Compute the co-occurance matrix of the corpus X
# Input: X - the tokenized sentences 
#        vocab - the dictionary which maps a word to its index in the words list
#        window_size - the size of the context window
def co_occ(X, vocab, window_size):
    n = len(vocab)
    matrix = np.zeros((n,n))
    for sentence in X:
        for i,word in enumerate(sentence):
            for j in range(max(i-window_size,0),min(i+window_size,len(sentence))):
                matrix[vocab[word]][sentence[j]]+=1
    return matrix


Note that Co-Occurrance matrix is not a word embedding that is generally used. Use PCA to decompose the matrix and reduce to K dimension. In addition, to maintain the semantic meaning of the word representation, K is preferred to be in the order of hundreds.

In [ ]:
co_matrix = co_occ(X, vocab, 5)

from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X_pca = pca.transform(co_matrix)

###**Part 4: Word2Vec**

Word2Vec is a popular technique of natural language processing to create word embeddings using neural networks, which can be further used for various text related tasks, e.g. sentiment anlaysis. In this problem, we will walk you through the implementation of Skip-Gram model, and the use of Gensim, a popular pacakge for Word2Vec. 
<br>   
In week 2 of 16ML, you have learned the mechanism of gradient descent as well as the implementation of stochastic gradient descent. In this problem, we will let you implement parts of the train function, including generating input and target data from raw tokenized sentences, and compute losses.

**(a)**
The training objective (for one training sample) is to maximize the conditional probability of observing the actual context word $w_{Oj}$ given the input context word $w_I$.
\begin{equation*}
\ P(w_{O,1}, w_{O,2}, ... , w_{O,C}| w_I) = \prod_{c=1}^C\frac{e^{u_{j^*_c}}}{\sum_{j'}^V e^{u_{j'}}} 
\end{equation*}
Where $u$ is the output layer before softmax, and $u_{j^*_c}$ is the node of $c^{th}$ context word in the output layer.
<br> Derive the loss function $E$ by taking the negative log of the above expression, and further prove that $$\frac{\partial E}{\partial u_j} = t_j - y_j$$

**(b)** The softmax function is a function that turns a vector of V real values into a vector of the same length that sums to 1, and each value is between 0 and 1, so that they can be interpreted as probabilities. Implement softmax function in the next cell. 
\begin{equation*}
\sigma(z)_i = \frac{e^{z_i}}{\sum_j^V e^{z_j}}
\end{equation*}



In [ ]:
def softmax(x):
    ## TODD:
    #  Compute softmax values for each sets of scores in x.
    return np.exp(x) / np.sum(np.exp(x), axis=0)

**(c)**
The following cell is the implementation of Word2Vec (Skip-Gram model) using stochastic gradient descent. Fill in the missing part labeled as TODD

In [ ]:
class word2vec(object): 
    def __init__(self, eta = 0.005, window_size = 2, hidden_num = 10): 
      # self.N - the number of neurons in the single hidden layer
      # self.words - the list of words
      # self.eta - the learning rate
      # self.vocab - the dictionary which maps the word to its index of self.words
      # self.V - the length of the words list
      # self.W1 - the first weight matrix of dimension |V| * N
      # self.W2 - the second weight matrix of dimension N * |V|
      # self.h - the hidden layer
      # self.u - the output layer

        self.N = hidden_num
        self.window_size = window_size
        self.eta = eta
        self.words = [] 
        self.vocab = {}
        self.V = None
        self.W1 = None
        self.W2 = None
        self.h = None
        self.u = None
        

    def initialize(self, X):
        # TODD
        # build the vocabulary from the tokenized sentences and update self.V, self.words
        data = {} 
        for tokens in X: 
            for word in tokens: 
                if word not in data: 
                    data[word] = 1
                else: 
                    data[word] += 1
                    
        self.V = len(data) 
        self.words = sorted(list(data.keys())) 
        
        for i in range(len(data)): 
            self.vocab[self.words[i]] = i 
   
        self.W1 = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
        self.W2 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 


    # TODD
    # Implement the forward function, update self.h and self.u, and return the result of softmax classification
    # X - the one hot vectors of center words
    def forward(self,X): 
        self.h = np.dot(self.W1.T,X).reshape(self.N,1) 
        self.u = np.dot(self.W2.T,self.h) 

        return softmax(self.u)   
  
    # TODD
    # Implement the backward function, update self.W1 and self.W2
    # input: X - the one hot vector of center word
    #        y - the output of softmax classification
    #        target - the representation of center word's context words in a single vector
    def backward(self,X,y,target): 
        du = y - np.array(target).reshape(-1,1) 
        dw2 = np.dot(self.h,du.T) 
        dh = np.dot(self.W2,du)
        dw1 = np.dot(np.array(X).reshape(-1,1), dh.T) 

        self.W2 = self.W2 - self.eta * dw2
        self.W1 = self.W1 - self.eta * dw1 
           
    def train(self,epochs, X):
        numTokens = len(sum(X, []))
        iterable = range(numTokens)
        itbar = tqdm(iterable)
        for e in epochs: 
            itbar.refresh()
            itbar.reset()
            self.loss = 0
            vocab = self.vocab

            for sentence in X: 
                for i in range(len(sentence)): 
                    itbar.update()
                    
                    ## TODD
                    # You need to process tokens of sentences (self.X) into valid training points
                    # 1. the input x (center_word) is an one hot encoding vector of the length of the vocabulary  
                    # 2. the output y (context_word) is of the same length of x, which has 1's in the sliding window of
                    #    the center_word except the center_word itself. Be catious of the boundary
                    
                    center_word = [0 for x in range(self.V)] 
                    center_word[vocab[sentence[i]]] = 1
                    context = [0 for x in range(self.V)] 

                    for j in range(i-w2v.window_size,i+w2v.window_size): 
                        if i!=j and j>=0 and j<len(sentence): 
                            context[vocab[sentence[j]]] += 1
                    
                    y = self.forward(center_word) 
                    self.backward(center_word, y, context) 
                    
                    ## TODD
                    # compute the loss based on the formula you derived
                    C = 0
                    for m in range(self.V): 
                        if(context[m]): 
                            self.loss += -1*self.u[m][0] 
                            C += 1
                    self.loss += C*np.log(np.sum(np.exp(self.u))) 

            print("epoch ",e+1, " loss = ",self.loss) 
            self.eta *= 1/( (1+self.eta*(e+1)) ) 

    # TODD
    # return context words of the given center word or None if word isn't in the vocabulary
    # Input: word - the center word
    #        num - the number of context words to return
    def predict(self,word,num): 
        if word in self.words: 
            X = [0 for i in range(self.V)] 
            X[self.vocab[word]] = 1
            prediction = self.forward(X) 
            output = {} 
            for i in range(self.V): 
                output[prediction[i][0]] = i 
               
            top_context_words = [] 
            for k in sorted(output,reverse=True): 
                top_context_words.append(self.words[output[k]]) 
                if(len(top_context_words)>=num): 
                    break
       
            return top_context_words 
        else: 
            return None


**(d)**
Because word2vec takes long time to train, you only use the following experiment to test your implementation. Run the next cell and train a word2vec embedding model using your implementation 

In [ ]:
X_w2v = X[:50]
w2v = word2vec() 
w2v.initialize(X_w2v)

# train 100 epochs, you should see your loss is contantly decreasing
epochs = tqdm(range(100))
w2v.train(epochs,X_w2v) 

Check your context words

In [ ]:
# check your context words
print(w2v.predict("good",3)) 

**(e)**
Gensim is a useful library to train your word2vec model faster. Run the following cell to train your Word2Vec model using gensim

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(X)

# model['word'] will give the embedding of the word "word"
model['word']

**(f)**
After you finish training your word2vec model with gensim, vectorize your tokenized sentences. For each sentence as a data point, we simply average the embedding of each word in this sentence. Implement the vectorize_word2vec function

In [ ]:
# Use trained word2vec model to vectorize the sentences. 
def vectorize_word2vec(X, model):
    ## TODD
    # Input: X - the tokenized sentences
    #        model - the trained word2vec model
    # return X_vectorized of dimension N * dim_embedding

    X_vectorized = []
    
    for i in range(len(X)):
        X_words = []
        for word in X[i]:
            if(word in model):
                X_words.append(model[word])
        X_vectorized.append(np.mean(np.array(X_words), axis = 0).reshape(-1))
    return np.array(X_vectorized)

X_vectorized = vectorize_word2vec(X, model)


Check the performance on the task of movie sentiment analysis with word2vec embedding

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size = 0.2)

from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))

**Beyond Word2vec: Capturing Word Context**